In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']= 'True'
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc


print('loading files..')

# boosting algorithm xgboost lightboost


train = pd.read_csv('./porto-seguro-safe-driver-prediction/train.csv' , na_values=-1)
test = pd.read_csv('./porto-seguro-safe-driver-prediction/test.csv' , na_values=-1)

col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)
test = test.drop(col_to_drop, axis=1)

loading files..


In [3]:
for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)

print(train.shape, test.shape)
train

(595212, 39) (892816, 38)


,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,7,0,2,2.0,5,1.0,0.0,0,1,0,...,1.0,0,0.0,1,12,2.0,0.400000,0.883679,0.370810,3.605551
1,9,0,1,1.0,7,0.0,0.0,0,0,1,...,1.0,1,2.0,1,19,3.0,0.316228,0.618816,0.388716,2.449490
2,13,0,5,4.0,9,1.0,0.0,0,0,1,...,1.0,1,2.0,1,60,1.0,0.316228,0.641586,0.347275,3.316625
3,16,0,0,1.0,2,0.0,0.0,1,0,0,...,1.0,1,3.0,1,104,1.0,0.374166,0.542949,0.294958,2.000000
4,17,0,0,2.0,0,1.0,0.0,1,0,0,...,1.0,1,2.0,1,82,3.0,0.316070,0.565831,0.365103,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1.0,10,0.0,0.0,0,0,0,...,1.0,1,0.0,1,31,3.0,0.374166,0.684631,0.385487,2.645751
595208,1488016,0,5,1.0,3,0.0,0.0,0,0,0,...,0.0,0,2.0,1,63,2.0,0.387298,0.972145,NaN,3.605551
595209,1488017,0,1,1.0,10,0.0,0.0,1,0,0,...,1.0,1,2.0,1,31,3.0,0.397492,0.596373,0.398748,1.732051
595210,1488021,0,5,2.0,3,1.0,0.0,0,0,1,...,1.0,1,2.0,1,101,3.0,0.374166,0.764434,0.384968,3.162278


In [4]:
# custom objective function (similar to auc)

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

In [6]:
# xgb
params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9,
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True}

X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values
sub=test['id'].to_frame()
sub['target']=0

In [7]:
nrounds = 200
kfold =2
# 원 데이터셋 그대로의 같은 비율로 데이터를 fold하여 뽑는다
skf = StratifiedKFold(n_splits=kfold, random_state=0,shuffle=True)

for i , (train_index, valid_index) in enumerate(skf.split(X,y)):
    break

In [8]:
train_index

array([     0,      3,      4, ..., 595207, 595210, 595211])

In [9]:
nrounds

200

In [10]:
X_train , X_valid  = X[train_index] , X[valid_index]

y_train, y_valid = y[train_index] , y[valid_index]


In [11]:
X_train

array([[ 2.        ,  2.        ,  5.        , ...,  0.8836789 ,
         0.3708099 ,  3.6055512 ],
       [ 0.        ,  1.        ,  2.        , ...,  0.5429488 ,
         0.29495764,  2.        ],
       [ 0.        ,  2.        ,  0.        , ...,  0.5658315 ,
         0.36510274,  2.        ],
       ...,
       [ 3.        ,  1.        , 10.        , ...,  0.6846306 ,
         0.3854867 ,  2.6457512 ],
       [ 5.        ,  2.        ,  3.        , ...,  0.7644341 ,
         0.38496754,  3.1622777 ],
       [ 0.        ,  1.        ,  8.        , ...,  0.9326493 ,
         0.37802115,  3.7416575 ]], dtype=float32)

In [ ]:
# xgb 전용 데이터 세트 numpy 입력 파라미터를 받는다
d_train = xgb.DMatrix(X_train, y_train)
d_valid = xgb.DMatrix(X_valid, y_valid)


In [ ]:
watchlist = [(d_train) , 'train') , (d_valid) ,'valid']

In [ ]:
# watchlist 를 활용하여 earlystopping 을 한다
# eval_metric 을 custimize function 인 gini_xgb를 사용
xgb_model = xgb.train(params, d_train , nrounds, watchlist , early_stopping_rounds=10, feval=gini_xgb , maximize=True, verbose_eval=100)

In [ ]:
# best에서 50번만 더 가서 예측해라
sub['target']  += xgb_model.predict(xgb.DMatrix(test[features].values),
                                        ntree_limit=xgb_model.best_ntree_limit+50 / (2*kfold))

In [ ]:
# xgboost를 2개의 fold로 나눠서 모델 2개로 0.5 percent를채웠다

# lightgbm 2개의 fold로 나눠서 모델 2개로 0.5 percent를채웠다
# 그러면 전체 1.0 submission 이나오고 두개를 stackking해서 제출한다

In [ ]:
xgb_model.best_ntree_limit_50


In [1]:
# 막간 연습
import heapq

# 머리 풀기 위해 알고리즘 한번씩

# 다익스트라의 핵심은 두 노드간의 최단 거리를 구하는건데 (그래야 응용 활용이 가능하다) 파라미터부터 외워버렸다
def dikstra(n,s,a,b,fares):

    graph = [{} for _ in range(n)]

    #노드의 개수만큼 distance 배열을 가지고 있다
    distances = [ float("inf") for i in range(n)]

    q = []
    # 최초의 거리 정보를 그래프 노드에 저장

    start , end , fare = fares[0][0] ,fares[0][1] , fares[0][2]

    for i in range(len(fares)):
        graph[fares[i][0]].append((fares[i][1], fares[i][2]))
        # 거꾸로도 넣어주어야함 해당 노드를 key 로 하여 정보를 추가하는 것임
        graph[fares[i][1]].append((fares[i][0], fares[i][2]))
    # a의 목적지와 시작지점, b의 목적지를 파라미터로 받고 리스트를 최소힙큐로 만들어 push하면
    # 부모노드가 항상 자식노드보다 값이 작은 큐가 되므로
    heapq.heappush(q, (s, a, b) )

    # 주어진 노드부터 목적지까지의 거리를 탐색하여 distances 배열에 업데이트한다 .
    #
    while q:
        start , a, b = q.heappop


        # 최단거리를 업데이트 그래프 노드를 탐색하며 순회하면서
        # start 지점을 경유


        for destination, cost in graph[start]:
            # distances[destination] == 목적지로 바로 가는 경우
            # distances[start]+ cost = start지점을 경유해서 가는 경우
            if distances[destination] > distances[start] + cost :


SyntaxError: unexpected EOF while parsing (3459626431.py, line 42)

In [ ]:
dic= {}

In [ ]:
dic[0] = {}

In [8]:
dic[0][0] = 1

KeyError: 0

In [14]:
dic[0][1] = 1

In [15]:
dic

{0: {1: 1}}

In [18]:
# 이차원 딕셔너리로 만든 그래프를 순회하는 것은 바깥 차원만 도는 것이므로
# 노드 개수만큼만 노드에 대한 거리를 초기화한다
{node : float('inf') for node in dic}

{0: inf}